# Working

In [5]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("src/")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:

from src.data import get_track , getBatch
from src.model import Transformer
import glob
import numpy as np

In [11]:
aPaths = glob.glob("data/*AllPSA.npy")
hpaths = glob.glob("data/*.hits.npy")
ap = np.random.choice(aPaths)
hp = np.random.choice(hpaths)
trk =  get_track(apth=ap,hpth=hp)
batch = getBatch(trk,batch_size=32)
X = batch[:,:,2:5]
Y = batch[:,:,5:]
model = Transformer()

# Experiment

In [43]:
predictions, loss = model(X,Y)

In [44]:
loss

<tf.Tensor: shape=(), dtype=float32, numpy=249.33801>

In [9]:
import numpy as np
import tensorflow as tf
from scipy.spatial import distance_matrix

N=200

xINP = np.random.uniform(-1,1,(N,2))
permutation = np.random.permutation(range(N))
yINP = np.array(list(map(lambda i: xINP[i], permutation)))
Noise = 0.01
Noise = 0.1
yINP += np.random.normal(0,Noise,(N,2))


def bipartete_matching(xINP:np.ndarray,yINP:np.ndarray)-> (np.ndarray,np.ndarray):
    cost_matrix = distance_matrix(xINP,yINP)

    queue = []
    for i in range(cost_matrix.shape[0]):
        for j in range(cost_matrix.shape[1]):
            queue.append([i,j,cost_matrix[i,j]])
    queue = sorted(queue, key=lambda x: x[2])

    row_to_col_match_vec = np.full(N,-1)
    col_to_row_match_vec = np.full(N,-1)
    for row in queue:
        if (row_to_col_match_vec[row[0]] == -1) & (col_to_row_match_vec[row[1]] == -1):
            row_to_col_match_vec[row[0]] = row[1]
            col_to_row_match_vec[row[1]] = row[0]
    return col_to_row_match_vec

In [10]:
class BipartateMatching(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
    def __call__(self,xINP:tf.Tensor,yINP:tf.Tensor)->tf.Tensor:
        cost_matrix = distance_matrix(xINP,yINP)

        queue = []
        for i in range(cost_matrix.shape[0]):
            for j in range(cost_matrix.shape[1]):
                queue.append([i,j,cost_matrix[i,j]])
        queue = sorted(queue, key=lambda x: x[2])

        row_to_col_match_vec = np.full(N,-1)
        col_to_row_match_vec = np.full(N,-1)
        for row in queue:
            if (row_to_col_match_vec[row[0]] == -1) & (col_to_row_match_vec[row[1]] == -1):
                row_to_col_match_vec[row[0]] = row[1]
                col_to_row_match_vec[row[1]] = row[0]
        return col_to_row_match_vec

In [11]:
lx = BipartateMatching()
lx(xINP,yINP)

TypeError: __call__() takes 2 positional arguments but 3 were given